# Starburst99 Tracks

This notebook specifically outputs the tracks to an Astropy ECSV file.

## Imports

In [ ]:
# Python Imports
from pathlib import Path
from tqdm.notebook import tqdm

In [ ]:
# Numerical Imports
import numpy as np

In [ ]:
# Astropy Collab Imports
from astropy import units as u
from astropy.table import QTable

from pysynphot import ObsBandpass

In [ ]:
# Plotting Imports
import matplotlib.pyplot as plt

In [ ]:
# Local Imports
from ssp import Starburst99Spectrum, CloudySpectrum

## Notebook Setup

In [ ]:
# Directories
TOP_DIR = Path('../../').resolve()

# Files
SB99_FN = TOP_DIR / 'DoctoralWork/Data/Starburst99/Sims/TailSim0_Genv00_014/TailSim0_Genv00_014.spectrum1'
CLDY_GLOB = TOP_DIR / 'DoctoralWork/Data/Cloudy/HII/Genv00_014_10Myr/rin_10pc/**/*.cont'

In [ ]:
# HST ACS-WFC Filters
HST_ACS_WFC_FILTERS = [
    'f435w', 'f475w', 'f502n', 'f550m', 'f555w', 'f606w', 'f625w', 'f658n',
    'f660n', 'f775w', 'f814w', 'f850lp', 'f892n'
]

# HST WFC3-UVIS Filters
HST_WFC3_UVIS_FILTERS = [
    'f200lp', 'f218w', 'f225w', 'f275w', 'f280n', 'f300x', 'f336w', 'f343n',
    'f350lp', 'f373n', 'f390m', 'f390w', 'f395n', 'f410m', 'f438w', 'f467m',
    'f469n', 'f475w', 'f475x', 'f487n', 'f502n', 'f547m', 'f555w', 'f600lp',
    'f606w', 'f621m', 'f625w', 'f631n', 'f645n', 'f656n', 'f657n', 'f658n',
    'f665n', 'f673n', 'f680n', 'f689m', 'f763m', 'f775w', 'f814w', 'f845m',
    'f850lp', 'f953n', 'fq232n', 'fq243n', 'fq378n', 'fq387n', 'fq422m',
    'fq436n', 'fq437n', 'fq492n', 'fq508n', 'fq575n', 'fq619n', 'fq634n',
    'fq672n', 'fq674n', 'fq727n', 'fq750n', 'fq889n', 'fq906n', 'fq924n',
    'fq937n'
]

# HST WFC3-IR Filters
HST_WFC3_IR_FILTERS = [
    'f098m', 'f105w', 'f110w', 'f125w', 'f126n', 'f127m', 'f128n', 'f130n',
    'f132n', 'f139m', 'f140w', 'f153m', 'f160w', 'f164n', 'f167n'
]

In [ ]:
# Redshift
REDSHIFT = 0.023997

# Distance to Source
LUMIN_DIST = 1.12e+2 * u.Mpc

## Load SSP Models

In [ ]:
# Load in Starburst99 and Mappings
sbMod = Starburst99Spectrum(
    str(SB99_FN), redshift=REDSHIFT, distToSrc=LUMIN_DIST.to_value('cm')
)
clMod = CloudySpectrum(
    str(CLDY_GLOB), redshift=REDSHIFT, distToSrc=LUMIN_DIST.to_value('cm')
)

# Trim Sb99 and Make Composite Timeline
stInd = np.flatnonzero(sbMod.years > 1e7)[0]  # 1e7 is ending year of Cloudy Model
sbMod.years = sbMod.years[stInd:]
sbMod.spectrumList = sbMod.spectrumList[stInd:]
model = clMod + sbMod

## Convolve with *HST* Filters and Output

In [ ]:
# Define the HST ACS Bandpass Filters
HST_BANDPASSES = {
    f'ACS-WFC-{filt.upper()}': ObsBandpass(f'acs,wfc1,{filt}')
    for filt in HST_ACS_WFC_FILTERS
}

# Add the WFC3-UVIS Filters
HST_BANDPASSES.update({
    f'WFC3-UVIS-{filt.upper()}': ObsBandpass(f'wfc3,uvis2,{filt}')
    for filt in HST_WFC3_UVIS_FILTERS
})

# Add the WFC3-IR Filters
HST_BANDPASSES.update({
    f'WFC3-IR-{filt.upper()}': ObsBandpass(f'wfc3,ir,{filt}')
    for filt in HST_WFC3_IR_FILTERS
})

In [ ]:
# Get the Magnitudes
modelMags = {}
modelMags['Age']   = model.years << u.yr
modelMags['Age'] <<= u.Myr
for obsMode, bandpass in tqdm(HST_BANDPASSES.items()):
    modelMags[obsMode] = model.makeobservation(bandpass) << u.ABmag

In [ ]:
# Covert to Astropy Table
modelTable = QTable(
    modelMags,
    meta={'redshift': REDSHIFT, 'luminosity distance': LUMIN_DIST}
)

# Save to ECSV
modelTable.write('Starburst99_Cloudy_Mags.ecsv', overwrite=True)
modelTable.write(
    'Starburst99_Cloudy_Mags.hdf5',
    path='data',
    serialize_meta=True,
    overwrite=True
)